In [37]:
import numpy as np
import pandas as pd
pd.__version__

'1.0.5'

In [122]:
#exercise information
exercisesList=['dumbbell curl','dumbbell kickback','barbell row','dumbbell shrug','kettlebell carry','lunge','kettlebell swing','high knees']
equipment=('dumbbell','barbell','kettlebell')
exercises=pd.Series(exercisesList,name='exercise')
print(exercises)
#routine information
size=pd.Series(['small','small','large','large','medium','large','large','small'])
reps=pd.Series([8,8,12,12,10,12,12,8])
sets=pd.Series(np.full(8,0))
routine=pd.DataFrame([size,reps,sets],index=['size','reps','sets'])

routine
routine=routine.append(exercises)

column=pd.MultiIndex.from_arrays([['upper body','upper body','upper body','upper body','upper body','lower body','lower body','lower body'],['biceps','triceps','lats','traps','core','quads','hamstrings','calves']],names=['muscle group','muscle'])
routine.columns=column
routine=routine.T
routine

0        dumbbell curl
1    dumbbell kickback
2          barbell row
3       dumbbell shrug
4     kettlebell carry
5                lunge
6     kettlebell swing
7           high knees
Name: exercise, dtype: object


size reps sets           exercise
muscle group muscle                                         
upper body   biceps       small    8    0      dumbbell curl
             triceps      small    8    0  dumbbell kickback
             lats         large   12    0        barbell row
             traps        large   12    0     dumbbell shrug
             core        medium   10    0   kettlebell carry
lower body   quads        large   12    0              lunge
             hamstrings   large   12    0   kettlebell swing
             calves       small    8    0         high knees

In [123]:
routine['reps']+=4
routine.sets+=3
routine['equipment']=[True,True,True,True,True,False,True,False]
print('Example exercise for each muscle\n',routine.iloc[:,3])
print('Updated to include an extra 4 reps, 3 sets, and whether equipment is used\n',routine)


Example exercise for each muscle
 muscle group  muscle    
upper body    biceps            dumbbell curl
              triceps       dumbbell kickback
              lats                barbell row
              traps            dumbbell shrug
              core           kettlebell carry
lower body    quads                     lunge
              hamstrings     kettlebell swing
              calves               high knees
Name: exercise, dtype: object
Updated to include an extra 4 reps, 3 sets, and whether equipment is used
                            size reps sets           exercise  equipment
muscle group muscle                                                    
upper body   biceps       small   12    3      dumbbell curl       True
             triceps      small   12    3  dumbbell kickback       True
             lats         large   16    3        barbell row       True
             traps        large   16    3     dumbbell shrug       True
             core        medium   14

In [124]:
routine['time']=pd.timedelta_range(0,periods=8,freq='4T15S')
print('Estimated start time of each workout',routine['time'],'\n')
repEquipment=routine.pivot_table('reps',index='equipment',columns='muscle',aggfunc='min')
print('Reps needed for each muscle and whether it uses equipment\n',repEquipment,'\n')
repEquipment=repEquipment.dropna(axis=0,thresh=3).fillna(0)
print('Adjusted to drop rows with too many null values and fill the others\n',repEquipment,'\n')
print('Series representation of the data\n',repEquipment.stack())
print('Averagee rep:',routine.reps.mean())
print('Minimum Reps for each muscle size:\n',routine.groupby('size')['reps'].sum())
print('Muscles and their corresponding exercises\n',routine.exercise.str.capitalize())


Estimated start time of each workout muscle group  muscle    
upper body    biceps       00:00:00
              triceps      00:04:15
              lats         00:08:30
              traps        00:12:45
              core         00:17:00
lower body    quads        00:21:15
              hamstrings   00:25:30
              calves       00:29:45
Name: time, dtype: timedelta64[ns] 

Reps needed for each muscle and whether it uses equipment
 muscle     biceps  calves  core  hamstrings  lats  quads  traps  triceps
equipment                                                               
False         NaN    12.0   NaN         NaN   NaN   16.0    NaN      NaN
True         12.0     NaN  14.0        16.0  16.0    NaN   16.0     12.0 

Adjusted to drop rows with too many null values and fill the others
 muscle     biceps  calves  core  hamstrings  lats  quads  traps  triceps
equipment                                                               
True         12.0     0.0  14.0        16.0  

In [125]:

#print(equipment)
#routine.exercise=routine.exercise.str.split()
print(routine,'\n')
print('Equipment used\n',*(string for string in routine.exercise.str.split().str.get(0) if string in equipment))
routine.reps=pd.eval((routine.reps-3)*routine.sets)
routine.sets=np.nan
print(routine)

                           size reps sets           exercise  equipment  \
muscle group muscle                                                       
upper body   biceps       small   12    3      dumbbell curl       True   
             triceps      small   12    3  dumbbell kickback       True   
             lats         large   16    3        barbell row       True   
             traps        large   16    3     dumbbell shrug       True   
             core        medium   14    3   kettlebell carry       True   
lower body   quads        large   16    3              lunge      False   
             hamstrings   large   16    3   kettlebell swing       True   
             calves       small   12    3         high knees      False   

                            time  
muscle group muscle               
upper body   biceps     00:00:00  
             triceps    00:04:15  
             lats       00:08:30  
             traps      00:12:45  
             core       00:17:00  
lowe

In [136]:
date=pd.to_datetime('May 15, 2021')
dates=date+pd.to_timedelta(np.arange(8),'D')
workouts=pd.DataFrame(dates,index=column,columns=['workout'])
print(workouts)
pd.merge(routine,workouts,left_index=True,right_index=True)

                           workout
muscle group muscle               
upper body   biceps     2021-05-15
             triceps    2021-05-16
             lats       2021-05-17
             traps      2021-05-18
             core       2021-05-19
lower body   quads      2021-05-20
             hamstrings 2021-05-21
             calves     2021-05-22


size reps  sets           exercise  equipment  \
muscle group muscle                                                        
upper body   biceps       small   27   NaN      dumbbell curl       True   
             triceps      small   27   NaN  dumbbell kickback       True   
             lats         large   39   NaN        barbell row       True   
             traps        large   39   NaN     dumbbell shrug       True   
             core        medium   33   NaN   kettlebell carry       True   
lower body   quads        large   39   NaN              lunge      False   
             hamstrings   large   39   NaN   kettlebell swing       True   
             calves       small   27   NaN         high knees      False   

                            time    workout  
muscle group muscle                          
upper body   biceps     00:00:00 2021-05-15  
             triceps    00:04:15 2021-05-16  
             lats       00:08:30 2021-05-17  
             traps      00:12:45 2021-05-18  
             core       00:17:00 2021-05-19  
lower body   quads      00:21:15 2021-05-20  
             hamstrings 00:25:30 2021-05-21  
             calves     00:29:45 2021-05-22